## SQL ##

Este es el codigo usado en sql para crear nuestra base de datos.

In [ ]:
"""
CREATE DATABASE IF NOT EXIST AIRBB
USE AIRBB

CREATE A TABLE DESCRIPTION_ (
Host_name VARCHAR (255),
Prices_per_night INT,
Beds INT,
Baths INT,
Cleaning_fees INT,
Maximum_guests INT,
PRIMARY KEY (Host_name)
);

CREATE A TABLE HOSTING (
urls VARCHAR(255),
Titles VARCHAR(255),
Property_types VARCHAR (255),
Host_name VARCHAR (255),
PRIMARY KEY (urls),
FOREING KEY (Host_name) REFERENCE DESCRIPTION_ (Host_name)
);

CREATE A TABLE COMENTS (
urls VARCHAR (255),
Ratings INT,
Num_reviews VARCHAR(255),
FOREING KEY (urls) REFERENCE HOSTING (urls)
);
"""

Este es el codigo para la subida a la base de datos

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv()
df #Importamos y vemo el df

Dividimos las columnas del df en distintos df segun las tablas de nuestra base de datos.

In [3]:
df_DESCRIPTION_ = df[['Host_name', 'Prices_per_night','Beds','Baths','Rooms','Cleaning_fees','Maximum_guests']]

In [4]:
df_HOSTING = df[['urls','Titles','Property_types','Host_name']]

In [8]:
df_STARS = df[['urls','Ratings','Num_reviews']]

Aui nos aseguramos de que no haya ningun nan sustituyendolos todos por none (valor que si admite sql).

In [ ]:
df_DESCRIPTION_ = df_DESCRIPTION_.replace({np.nan : None}) #Sustituimos todos los posibles nan por none para poder subirlos
df_HOSTING = df_HOSTING.replace({np.nan : None})
df_STARS = df_STARS.replace({np.nan : None})

Aqui procedemos a la subida de las distintas tablas.

Description_

In [ ]:
column_names= ['Host_name', 'Prices_per_night','Beds','Baths','Rooms','Cleaning_fees','Maximum_guests']

In [ ]:
database = "AIRBB"
table_name = "DESCRIPTION_"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="password123",
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_DESCRIPTION_.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_DESCRIPTION_[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Hosting

In [ ]:
column_names= ['urls','Titles','Property_types','Host_name']

In [ ]:
database = "AIRBB"
table_name = "HOSTING"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="password123",
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_HOSTING.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_HOSTING[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()

Stars

In [ ]:
column_names= ['urls','Ratings','Num_reviews']

In [ ]:
database = "AIRBB"
table_name = "STARS"

db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="password123",
                             database=database,
                             consume_results = True)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names

# Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
if len(df_HOSTING.columns) != len(column_names):
    raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")

# Preparar la consulta de inserción
insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"

# Convertir las filas del DataFrame a tuplas
values = [tuple(row) for row in df_HOSTING[list(column_names)].values]

# Ejecutar la consulta de inserción
cursor.executemany(insert_query, values)

# Confirmar los cambios en la base de datos
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

# Vaciamos el cursor
cursor.close()
db.close()